### 載入模型

In [1]:
import pickle

# load the trained decision tree classifier from Pickle
decision_tree_1_pkl_filename = 'decision_tree_1.pkl'
# Open the file to load as pkl file
with open(file=decision_tree_1_pkl_filename, mode='rb') as f:
    decision_tree_1=pickle.load(f)
    
decision_tree_2_pkl_filename = 'decision_tree_2.pkl'
# Open the file to load as pkl file
with open(file=decision_tree_2_pkl_filename, mode='rb') as f:
    decision_tree_2=pickle.load(f)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### 產生C code

In [2]:
from sklearn.tree import DecisionTreeClassifier, _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            #threshold = tree_.threshold[node]
            threshold = np.round(tree_.threshold[node],decimals=4)
            left_child = tree_.children_left[node]
            right_child = tree_.children_right[node]
            yield f"{indent}if (X[{name}] <= {threshold}) {{"
            for line in recurse(left_child, depth + 1):
                yield line
            yield f"{indent}}} else {{"
            for line in recurse(right_child, depth + 1):
                yield line
            yield f"{indent}}}"
        else:
            yield f"{indent}return {np.round(tree_.value[node][0][0],decimals=3)};"
    
    code = "\n".join(recurse(0, 1))
    return code

def genC(my_tree, fName):
    feature_names=['0','1','2','3','4','5','6','7','8','9',
               '10','11','12','13','14','15','16','17','18','19',
               '20','21','22','23','24','25','26','27','29','29']
    code = tree_to_code(my_tree, feature_names)
    with open(fName, "w") as f:
        f.write(f'''
#include <stdio.h>

float predict(int X[]) {{
{code}
}}

int main() {{
  int X[] = {{1, 2, 3, 4}};
  printf("Predicted class: %f", predict(X));
}}
''')
        
genC(decision_tree_1,"decision_tree_1.c")
genC(decision_tree_2,"decision_tree_2.c")


NameError: name 'np' is not defined